In [ ]:
!gdown 1se3BrRr2M82zN7sf-52OM_BYyt4gZVpK
!unzip P03_6000_train.zip -d P03_6000_train

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm,trange
from sklearn.model_selection import train_test_split
import sklearn.metrics
import imageio

In [ ]:
df = pd.read_csv('/content/P03_1000_test/test.csv', header=None)
df.columns = ['image_id']

In [ ]:
df['image_id'] = df['image_id'].str.replace('.gif', '')

<ipython-input-5-87273b52bf7b>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['image_id'] = df['image_id'].str.replace('.gif', '')


In [ ]:
df

,image_id
0,605000_6478000
1,549000_6579000
2,639000_6432000
3,625000_6400000
4,409000_6448000
...,...
995,594000_6499000
996,575000_6459000
997,421000_6528000
998,412000_6536000


In [ ]:
final_df = pd.DataFrame(columns=['image_id', 'xmin', 'ymin', 'width', 'height', 'centerx', 'centery'])

In [ ]:
INTERSECTION_SIZE = 30
UPPER_LIMIT = 999
LOWER_LIMIT = 0

def create_bbox(center_x, center_y, width, height):
  xmin = center_x - width / 2
  ymin = center_y - height / 2

  if xmin < LOWER_LIMIT:
    width = width - np.absolute(xmin)
    xmin = LOWER_LIMIT

  elif xmin + width > UPPER_LIMIT:
    width = width - (xmin + width - UPPER_LIMIT)

  if ymin < LOWER_LIMIT:
    height = height - np.absolute(ymin)
    ymin = LOWER_LIMIT

  elif ymin + height > UPPER_LIMIT:
    height = height - (ymin + height - UPPER_LIMIT)

  return (xmin, ymin, width, height)

In [ ]:
with open("ristmikud.csv") as f:
  for n, line in enumerate(f):
    line = line.strip().split(',')
    image = line.pop(0)
    if any(df['image_id'] == image):
      for i, intersection in enumerate(line):
        inter_coords = intersection.strip().split('-')
        inter_coords = [int(x) for x in inter_coords]
        (xmin, ymin, width, height) = create_bbox(inter_coords[0], inter_coords[1], INTERSECTION_SIZE, INTERSECTION_SIZE)

        new_row_df = pd.DataFrame([{'image_id': image, 'xmin': xmin, 'ymin': ymin, 'width': width, 'height': height, 'centerx': inter_coords[0], 'centery':inter_coords[1]}])
        final_df = pd.concat([final_df, new_row_df], ignore_index=True)

In [ ]:
final_df

,image_id,xmin,ymin,width,height,centerx,centery
0,376000_6474000,265.0,153.0,30,30,280,168
1,376000_6474000,244.0,235.0,30,30,259,250
2,376000_6474000,308.0,163.0,30,30,323,178
3,376000_6474000,350.0,123.0,30,30,365,138
4,376000_6474000,391.0,82.0,30,30,406,97
...,...,...,...,...,...,...,...
7040,717000_6427000,259.0,515.0,30,30,274,530
7041,717000_6427000,293.0,640.0,30,30,308,655
7042,717000_6427000,232.0,745.0,30,30,247,760
7043,717000_6427000,281.0,773.0,30,30,296,788


In [ ]:
unique_count = final_df['image_id'].nunique()
unique_count

672

In [ ]:
final_df.to_csv('test_gt.csv', index=False)

In [ ]:
del final_df